# 정보처리연습 기말 보고서 코드 - 유혜영 (2016195045)

# Step 1. Data Collection

In [1]:
# Import necessary libraries
from bs4 import BeautifulSoup
import lxml
from urllib.request import urlopen
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

## 1. Expedia

In [126]:
# Create an empty list to store expedia review urls
expedia_review_urls = []
# Create a url for each expedia review page (there are 107)
for x in range (0,107):
    url = "https://www.trustpilot.com/review/www.expedia.com?languages=en&page="+str(x+1)
    # Append the urls to the expedia_review_urls list
    expedia_review_urls.append(url)
    #print(url)
# Print the number of urls in the expedia_review_urls list
print(len(expedia_review_urls))

107


In [127]:
# Process indicator
process = "Crawling webpage: "

# Making empty dictionary for expedia reviews
expedia_reviews = {}

# Making empty lists for expedia review components
expedia_review_contents = []
expedia_review_scores = []
expedia_review_numbers = []
expedia_review_weekdays = []

# For loop for scraping data for expedia_review_urls
for url in expedia_review_urls:
    
    html = urlopen(url)
    soup = BeautifulSoup(html,"html.parser")
    
    # Review content
    review_content_infos = soup.find_all("p",{"class":"review-info__body__text"})
    for review_content_info in review_content_infos:
        review_content_info_str = str(review_content_info)
        review_content = review_content_info_str.split('body__text">')[1].split('</p>')[0].replace("\n","").replace("<br/>","")
        expedia_review_contents.append(review_content)
    
    # Review score
    review_score_infos = soup.find_all("script",{"data-initial-state":"review-info"})
    for review_score_info in review_score_infos:
        review_score = review_score_info.text.split('stars":')[1].split("}")[0]
        expedia_review_scores.append(review_score)

    # Number of reviews the reviewer wrote
    review_number_infos = soup.find_all("span",{"class":"consumer-info__details__review-count"})
    for review_number_info in review_number_infos:
        review_number_info_str = str(review_number_info)
        review_number = review_number_info_str.split('review-count">')[1].split(' review')[0].replace("\n","")
        expedia_review_numbers.append(review_number)
    
    # Weekday of review
    review_date_infos = soup.find_all("div",{"class":"header__verified__date"})
    for review_date_info in review_date_infos:
        review_date_info_str = str(review_date_info)
        review_weekday = review_date_info_str.split('title="')[1].split(",")[0]
        expedia_review_weekdays.append(review_weekday)

    
    # Print process to know how your crawler is doing
    print(process + str(url))
    
# Add all the information collected in the four lists to the dictionary
expedia_reviews["review_contents"] = expedia_review_contents
expedia_reviews["review_numbers"] = expedia_review_numbers
expedia_reviews["review_scores"] = expedia_review_scores
expedia_reviews["review_weekdays"] = expedia_review_weekdays

# Add the company name "expedia" to company_name
expedia_reviews["company_name"] = "expedia"

Crawling webpage: https://www.trustpilot.com/review/www.expedia.com?languages=en&page=1
Crawling webpage: https://www.trustpilot.com/review/www.expedia.com?languages=en&page=2
Crawling webpage: https://www.trustpilot.com/review/www.expedia.com?languages=en&page=3
Crawling webpage: https://www.trustpilot.com/review/www.expedia.com?languages=en&page=4
Crawling webpage: https://www.trustpilot.com/review/www.expedia.com?languages=en&page=5
Crawling webpage: https://www.trustpilot.com/review/www.expedia.com?languages=en&page=6
Crawling webpage: https://www.trustpilot.com/review/www.expedia.com?languages=en&page=7
Crawling webpage: https://www.trustpilot.com/review/www.expedia.com?languages=en&page=8
Crawling webpage: https://www.trustpilot.com/review/www.expedia.com?languages=en&page=9
Crawling webpage: https://www.trustpilot.com/review/www.expedia.com?languages=en&page=10
Crawling webpage: https://www.trustpilot.com/review/www.expedia.com?languages=en&page=11
Crawling webpage: https://www.

Crawling webpage: https://www.trustpilot.com/review/www.expedia.com?languages=en&page=94
Crawling webpage: https://www.trustpilot.com/review/www.expedia.com?languages=en&page=95
Crawling webpage: https://www.trustpilot.com/review/www.expedia.com?languages=en&page=96
Crawling webpage: https://www.trustpilot.com/review/www.expedia.com?languages=en&page=97
Crawling webpage: https://www.trustpilot.com/review/www.expedia.com?languages=en&page=98
Crawling webpage: https://www.trustpilot.com/review/www.expedia.com?languages=en&page=99
Crawling webpage: https://www.trustpilot.com/review/www.expedia.com?languages=en&page=100
Crawling webpage: https://www.trustpilot.com/review/www.expedia.com?languages=en&page=101
Crawling webpage: https://www.trustpilot.com/review/www.expedia.com?languages=en&page=102
Crawling webpage: https://www.trustpilot.com/review/www.expedia.com?languages=en&page=103
Crawling webpage: https://www.trustpilot.com/review/www.expedia.com?languages=en&page=104
Crawling webpage

In [133]:
# Check the number of data in each column to see if there is any data that does not match up
print(len(expedia_review_weekdays))
print(len(expedia_review_scores))
print(len(expedia_review_contents))
print(len(expedia_review_numbers))

2123
2123
2123
2123


In [134]:
# Assign SentimentIntensityAnalyzer() to variable analyzer
analyzer = SentimentIntensityAnalyzer()

# Create an empty list that is labeled expedia_compounds to collect valence values
expedia_compounds = []

# For loop to get the polarity scores of the review content and the valence
for expedia_review_content in expedia_review_contents:
    # Polarity score of the review content
    vs = analyzer.polarity_scores(expedia_review_content)
    # Determining valence of review content
    if vs['compound'] >= 0.5:
        expedia_compounds.append("positive")
    elif vs['compound'] <= -0.5:
        expedia_compounds.append("negative")
    else:
        expedia_compounds.append("neutral")

# Assign the valence values to the key sentiment_compound
expedia_reviews["sentiment_compound"] = expedia_compounds
# Check the number of data in expedia_compounds to check if it matches with review_contents
print(len(expedia_compounds))

2123


In [135]:
# Make a dataframe with the expedia_reviews dictionary
expedia_df = pd.DataFrame(expedia_reviews)
expedia_df

,review_contents,review_numbers,review_scores,review_weekdays,company_name,sentiment_compound
0,I will never book through Expedia ...,1,1,Monday,expedia,neutral
1,I really regret and feel bad that ...,1,1,Sunday,expedia,neutral
2,"Don't trust this company at all, d...",1,1,Sunday,expedia,negative
3,"Booked a Vacation Package all ok, ...",1,1,Friday,expedia,negative
4,I've booked with Expedia bunch of ...,2,3,Friday,expedia,negative
5,Worst customer service I have ever...,1,1,Friday,expedia,negative
6,Will never make reservations throu...,1,1,Thursday,expedia,neutral
7,WHY IT PAYS NOT TO BOOK HOTEL ACCO...,4,1,Sunday,expedia,positive
8,"flew to usa,on arrival discovered ...",1,1,Friday,expedia,neutral
9,WORST travel website I advise you ...,1,1,Thursday,expedia,negative


In [136]:
# Save expedia_reviews dataframe to tsv file on laptop
expedia_df.to_csv("/Users/angieryu2202/Desktop/expedia_reviews_tp.tsv", sep="\t")

## 2. Booking.com

In [137]:
# Create an empty list to store booking review urls
booking_review_urls = []
# Create a url for each booking review page (although there are actually 171 pages, downsampled to 107 to sort out the imbalance)
for x in range (0,107):
    url = "https://www.trustpilot.com/review/www.booking.com?languages=en&page="+str(x+1)
    # Append the urls to the booking_review_urls list
    booking_review_urls.append(url)
    #print(url)
# Print the number of urls in the booking_review_urls list
print(len(booking_review_urls))

107


In [138]:
# Process indicator
process = "Crawling webpage: "

# Making empty dictionary for booking reviews
booking_reviews = {}

# Making empty lists for booking review components
booking_review_contents = []
booking_review_scores = []
booking_review_numbers = []
booking_review_weekdays = []

for url in booking_review_urls:
    
    html = urlopen(url)
    soup = BeautifulSoup(html,"html.parser")
    
    # Review content
    review_content_infos = soup.find_all("p",{"class":"review-info__body__text"})
    for review_content_info in review_content_infos:
        review_content_info_str = str(review_content_info)
        review_content = review_content_info_str.split('body__text">')[1].split('</p>')[0].replace("\n","").replace("<br/>","")
        booking_review_contents.append(review_content)
    
    # Review score
    review_score_infos = soup.find_all("script",{"data-initial-state":"review-info"})
    for review_score_info in review_score_infos:
        review_score = review_score_info.text.split('stars":')[1].split("}")[0]
        booking_review_scores.append(review_score)

    # Number of reviews the reviewer wrote
    review_number_infos = soup.find_all("span",{"class":"consumer-info__details__review-count"})
    for review_number_info in review_number_infos:
        review_number_info_str = str(review_number_info)
        review_number = review_number_info_str.split('review-count">')[1].split(' review')[0].replace("\n","")
        booking_review_numbers.append(review_number)
    
    # Weekday of review
    review_date_infos = soup.find_all("div",{"class":"header__verified__date"})
    for review_date_info in review_date_infos:
        review_date_info_str = str(review_date_info)
        review_weekday = review_date_info_str.split('title="')[1].split(",")[0]
        booking_review_weekdays.append(review_weekday)

    
    # Print process to know how your crawler is doing
    print(process + str(url))
    
# Add all the information collected in the four lists to the dictionary
booking_reviews["review_contents"] = booking_review_contents
booking_reviews["review_numbers"] = booking_review_numbers
booking_reviews["review_scores"] = booking_review_scores
booking_reviews["review_weekdays"] = booking_review_weekdays

# Add the company name "booking" to company_name
booking_reviews["company_name"] = "booking"

Crawling webpage: https://www.trustpilot.com/review/www.booking.com?languages=en&page=1
Crawling webpage: https://www.trustpilot.com/review/www.booking.com?languages=en&page=2
Crawling webpage: https://www.trustpilot.com/review/www.booking.com?languages=en&page=3
Crawling webpage: https://www.trustpilot.com/review/www.booking.com?languages=en&page=4
Crawling webpage: https://www.trustpilot.com/review/www.booking.com?languages=en&page=5
Crawling webpage: https://www.trustpilot.com/review/www.booking.com?languages=en&page=6
Crawling webpage: https://www.trustpilot.com/review/www.booking.com?languages=en&page=7
Crawling webpage: https://www.trustpilot.com/review/www.booking.com?languages=en&page=8
Crawling webpage: https://www.trustpilot.com/review/www.booking.com?languages=en&page=9
Crawling webpage: https://www.trustpilot.com/review/www.booking.com?languages=en&page=10
Crawling webpage: https://www.trustpilot.com/review/www.booking.com?languages=en&page=11
Crawling webpage: https://www.

Crawling webpage: https://www.trustpilot.com/review/www.booking.com?languages=en&page=94
Crawling webpage: https://www.trustpilot.com/review/www.booking.com?languages=en&page=95
Crawling webpage: https://www.trustpilot.com/review/www.booking.com?languages=en&page=96
Crawling webpage: https://www.trustpilot.com/review/www.booking.com?languages=en&page=97
Crawling webpage: https://www.trustpilot.com/review/www.booking.com?languages=en&page=98
Crawling webpage: https://www.trustpilot.com/review/www.booking.com?languages=en&page=99
Crawling webpage: https://www.trustpilot.com/review/www.booking.com?languages=en&page=100
Crawling webpage: https://www.trustpilot.com/review/www.booking.com?languages=en&page=101
Crawling webpage: https://www.trustpilot.com/review/www.booking.com?languages=en&page=102
Crawling webpage: https://www.trustpilot.com/review/www.booking.com?languages=en&page=103
Crawling webpage: https://www.trustpilot.com/review/www.booking.com?languages=en&page=104
Crawling webpage

In [139]:
# Check the number of data in each column to see if there is any data that does not match up
print(len(booking_review_weekdays))
print(len(booking_review_scores))
print(len(booking_review_contents))
print(len(booking_review_numbers))

2140
2140
2140
2140


In [141]:
# Assign SentimentIntensityAnalyzer() to variable analyzer
analyzer = SentimentIntensityAnalyzer()

# Create an empty list that is labeled booking_compounds to collect valence values
booking_compounds = []

# For loop to get the polarity scores of the review content and the valence
for booking_review_content in booking_review_contents:
    # Polarity score of the review content
    vs = analyzer.polarity_scores(booking_review_content)
    # Determining valence of review content
    if vs['compound'] >= 0.5:
        booking_compounds.append("positive")
    elif vs['compound'] <= -0.5:
        booking_compounds.append("negative")
    else:
        booking_compounds.append("neutral")

# Assign the valence values to the key sentiment_compound
booking_reviews["sentiment_compound"] = booking_compounds
# Check the number of data in booking_compounds to check if it matches with review_contents
print(len(booking_compounds))

2140


In [142]:
# Make a dataframe with the booking_reviews dictionary
booking_df = pd.DataFrame(booking_reviews)
booking_df

,review_contents,review_numbers,review_scores,review_weekdays,company_name,sentiment_compound
0,Booked a hotel through booking.com...,1,1,Tuesday,booking,positive
1,The entire platform is extremely e...,3,5,Monday,booking,positive
2,"Unfortunately, you must very caref...",1,2,Sunday,booking,positive
3,Only used them once and very dissa...,6,1,Sunday,booking,neutral
4,Please be aware if your flight is ...,9,1,Saturday,booking,neutral
5,We booked milford sound trip from ...,1,1,Saturday,booking,negative
6,I feel like I've been lied to that...,1,1,Friday,booking,negative
7,Can't say a bad word about them re...,1,5,Friday,booking,positive
8,no hot water no response to the co...,1,1,Thursday,booking,negative
9,I'm surprised with seeing booking....,7,5,Thursday,booking,positive


In [143]:
# Save booking_reviews dataframe to tsv file on laptop
booking_df.to_csv("/Users/angieryu2202/Desktop/booking_reviews_tp.tsv", sep="\t")

## 3. Agoda

In [144]:
# Create an empty list to store agoda review urls
agoda_review_urls = []
# Create a url for each agoda review page (although there are actually 866 pages, downsampled to 107 to sort out the imbalance)
for x in range (0,107):
    url = "https://www.trustpilot.com/review/www.agoda.com?languages=en&page="+str(x+1)
    # Append the urls to the agoda_review_urls list
    agoda_review_urls.append(url)
    #print(url)
# Print the number of urls in the agoda_review_urls list
print(len(agoda_review_urls))

107


In [145]:
# Process indicator
process = "Crawling webpage: "

# Making empty dictionary for agoda reviews
agoda_reviews = {}

# Making empty lists for agoda review components
agoda_review_contents = []
agoda_review_scores = []
agoda_review_numbers = []
agoda_review_weekdays = []

for url in agoda_review_urls:
    
    html = urlopen(url)
    soup = BeautifulSoup(html,"html.parser")
    
    # Review content
    review_content_infos = soup.find_all("p",{"class":"review-info__body__text"})
    for review_content_info in review_content_infos:
        review_content_info_str = str(review_content_info)
        review_content = review_content_info_str.split('body__text">')[1].split('</p>')[0].replace("\n","").replace("<br/>","")
        agoda_review_contents.append(review_content)
    
    # Review score
    review_score_infos = soup.find_all("script",{"data-initial-state":"review-info"})
    for review_score_info in review_score_infos:
        review_score = review_score_info.text.split('stars":')[1].split("}")[0]
        agoda_review_scores.append(review_score)

    # Number of reviews the reviewer wrote
    review_number_infos = soup.find_all("span",{"class":"consumer-info__details__review-count"})
    for review_number_info in review_number_infos:
        review_number_info_str = str(review_number_info)
        review_number = review_number_info_str.split('review-count">')[1].split(' review')[0].replace("\n","")
        agoda_review_numbers.append(review_number)
    
    # Weekday of review
    review_date_infos = soup.find_all("div",{"class":"header__verified__date"})
    for review_date_info in review_date_infos:
        review_date_info_str = str(review_date_info)
        review_weekday = review_date_info_str.split('title="')[1].split(",")[0]
        agoda_review_weekdays.append(review_weekday)

    
    # Print process to know how your crawler is doing
    print(process + str(url))
    
# Add all the information collected in the four lists to the dictionary
agoda_reviews["review_contents"] = agoda_review_contents
agoda_reviews["review_numbers"] = agoda_review_numbers
agoda_reviews["review_scores"] = agoda_review_scores
agoda_reviews["review_weekdays"] = agoda_review_weekdays

# Add the company name "agoda" to company_name
agoda_reviews["company_name"] = "agoda"

Crawling webpage: https://www.trustpilot.com/review/www.agoda.com?languages=en&page=1
Crawling webpage: https://www.trustpilot.com/review/www.agoda.com?languages=en&page=2
Crawling webpage: https://www.trustpilot.com/review/www.agoda.com?languages=en&page=3
Crawling webpage: https://www.trustpilot.com/review/www.agoda.com?languages=en&page=4
Crawling webpage: https://www.trustpilot.com/review/www.agoda.com?languages=en&page=5
Crawling webpage: https://www.trustpilot.com/review/www.agoda.com?languages=en&page=6
Crawling webpage: https://www.trustpilot.com/review/www.agoda.com?languages=en&page=7
Crawling webpage: https://www.trustpilot.com/review/www.agoda.com?languages=en&page=8
Crawling webpage: https://www.trustpilot.com/review/www.agoda.com?languages=en&page=9
Crawling webpage: https://www.trustpilot.com/review/www.agoda.com?languages=en&page=10
Crawling webpage: https://www.trustpilot.com/review/www.agoda.com?languages=en&page=11
Crawling webpage: https://www.trustpilot.com/review/

Crawling webpage: https://www.trustpilot.com/review/www.agoda.com?languages=en&page=96
Crawling webpage: https://www.trustpilot.com/review/www.agoda.com?languages=en&page=97
Crawling webpage: https://www.trustpilot.com/review/www.agoda.com?languages=en&page=98
Crawling webpage: https://www.trustpilot.com/review/www.agoda.com?languages=en&page=99
Crawling webpage: https://www.trustpilot.com/review/www.agoda.com?languages=en&page=100
Crawling webpage: https://www.trustpilot.com/review/www.agoda.com?languages=en&page=101
Crawling webpage: https://www.trustpilot.com/review/www.agoda.com?languages=en&page=102
Crawling webpage: https://www.trustpilot.com/review/www.agoda.com?languages=en&page=103
Crawling webpage: https://www.trustpilot.com/review/www.agoda.com?languages=en&page=104
Crawling webpage: https://www.trustpilot.com/review/www.agoda.com?languages=en&page=105
Crawling webpage: https://www.trustpilot.com/review/www.agoda.com?languages=en&page=106
Crawling webpage: https://www.trustp

In [146]:
# Check the number of data in each column to see if there is any data that does not match up
print(len(agoda_review_weekdays))
print(len(agoda_review_scores))
print(len(agoda_review_contents))
print(len(agoda_review_numbers))

2140
2140
2140
2140


In [147]:
# Assign SentimentIntensityAnalyzer() to variable analyzer
analyzer = SentimentIntensityAnalyzer()

# Create an empty list that is labeled agoda_compounds to collect valence values
agoda_compounds = []

# For loop to get the polarity scores of the review content and the valence
for agoda_review_content in agoda_review_contents:
    # Polarity score of the review content
    vs = analyzer.polarity_scores(agoda_review_content)
    # Determining valence of review content
    if vs['compound'] >= 0.5:
        agoda_compounds.append("positive")
    elif vs['compound'] <= -0.5:
        agoda_compounds.append("negative")
    else:
        agoda_compounds.append("neutral")

# Assign the valence values to the key sentiment_compound
agoda_reviews["sentiment_compound"] = agoda_compounds
# Check the number of data in agoda_compounds to check if it matches with review_contents
print(len(agoda_compounds))

2140


In [148]:
# Make a dataframe with the agoda_reviews dictionary
agoda_df = pd.DataFrame(agoda_reviews)
agoda_df

,review_contents,review_numbers,review_scores,review_weekdays,company_name,sentiment_compound
0,Fast and easy.,1,5,Monday,agoda,neutral
1,They charged my card 4 days before...,1,1,Monday,agoda,negative
2,Always best prices,9,5,Monday,agoda,positive
3,the parking is not free,1,3,Monday,agoda,neutral
4,nice review!!,1,5,Monday,agoda,positive
5,Agoda is helful in settle its prog...,1,4,Monday,agoda,neutral
6,Agoda made an error with my bookin...,2,1,Monday,agoda,negative
7,Agoda is the best,1,5,Monday,agoda,positive
8,Easy to book a room.,1,5,Sunday,agoda,neutral
9,Very Good place,1,5,Sunday,agoda,neutral


In [149]:
# Save agoda_reviews dataframe to tsv file on laptop
agoda_df.to_csv("/Users/angieryu2202/Desktop/agoda_reviews_tp.tsv", sep="\t")

## 4. Priceline

In [3]:
# Create an empty list to store priceline review urls
priceline_review_urls = []
# Create a url for each priceline review page (although there are actually 2682 pages, downsampled to 107 to sort out the imbalance)
for x in range (0,107):
    url = "https://www.trustpilot.com/review/www.priceline.com?languages=en&page="+str(x+1)
    # Append the urls to the priceline_review_urls list
    priceline_review_urls.append(url)
    #print(url)
# Print the number of urls in the priceline_review_urls list
print(len(priceline_review_urls))

107


In [4]:
# Process indicator
process = "Crawling webpage: "

# Making empty dictionary for priceline reviews
priceline_reviews = {}

# Making empty lists for priceline review components
priceline_review_contents = []
priceline_review_scores = []
priceline_review_numbers = []
priceline_review_weekdays = []

for url in priceline_review_urls:
    
    html = urlopen(url)
    soup = BeautifulSoup(html,"html.parser")
    
    # Review content
    review_content_infos = soup.find_all("p",{"class":"review-info__body__text"})
    for review_content_info in review_content_infos:
        review_content_info_str = str(review_content_info)
        review_content = review_content_info_str.split('body__text">')[1].split('</p>')[0].replace("\n","").replace("<br/>","")
        priceline_review_contents.append(review_content)
    
    # Review score
    review_score_infos = soup.find_all("script",{"data-initial-state":"review-info"})
    for review_score_info in review_score_infos:
        review_score = review_score_info.text.split('stars":')[1].split("}")[0]
        priceline_review_scores.append(review_score)

    # Number of reviews the reviewer wrote
    review_number_infos = soup.find_all("span",{"class":"consumer-info__details__review-count"})
    for review_number_info in review_number_infos:
        review_number_info_str = str(review_number_info)
        review_number = review_number_info_str.split('review-count">')[1].split(' review')[0].replace("\n","")
        priceline_review_numbers.append(review_number)
    
    # Weekday of review
    review_date_infos = soup.find_all("div",{"class":"header__verified__date"})
    for review_date_info in review_date_infos:
        review_date_info_str = str(review_date_info)
        review_weekday = review_date_info_str.split('title="')[1].split(",")[0]
        priceline_review_weekdays.append(review_weekday)

    
    # Print process to know how your crawler is doing
    print(process + str(url))
    
# Add all the information collected in the four lists to the dictionary
priceline_reviews["review_contents"] = priceline_review_contents
priceline_reviews["review_numbers"] = priceline_review_numbers
priceline_reviews["review_scores"] = priceline_review_scores
priceline_reviews["review_weekdays"] = priceline_review_weekdays

# Add the company name "priceline" to company_name
priceline_reviews["company_name"] = "priceline"

Crawling webpage: https://www.trustpilot.com/review/www.priceline.com?languages=en&page=1
Crawling webpage: https://www.trustpilot.com/review/www.priceline.com?languages=en&page=2
Crawling webpage: https://www.trustpilot.com/review/www.priceline.com?languages=en&page=3
Crawling webpage: https://www.trustpilot.com/review/www.priceline.com?languages=en&page=4
Crawling webpage: https://www.trustpilot.com/review/www.priceline.com?languages=en&page=5
Crawling webpage: https://www.trustpilot.com/review/www.priceline.com?languages=en&page=6
Crawling webpage: https://www.trustpilot.com/review/www.priceline.com?languages=en&page=7
Crawling webpage: https://www.trustpilot.com/review/www.priceline.com?languages=en&page=8
Crawling webpage: https://www.trustpilot.com/review/www.priceline.com?languages=en&page=9
Crawling webpage: https://www.trustpilot.com/review/www.priceline.com?languages=en&page=10
Crawling webpage: https://www.trustpilot.com/review/www.priceline.com?languages=en&page=11
Crawling

Crawling webpage: https://www.trustpilot.com/review/www.priceline.com?languages=en&page=92
Crawling webpage: https://www.trustpilot.com/review/www.priceline.com?languages=en&page=93
Crawling webpage: https://www.trustpilot.com/review/www.priceline.com?languages=en&page=94
Crawling webpage: https://www.trustpilot.com/review/www.priceline.com?languages=en&page=95
Crawling webpage: https://www.trustpilot.com/review/www.priceline.com?languages=en&page=96
Crawling webpage: https://www.trustpilot.com/review/www.priceline.com?languages=en&page=97
Crawling webpage: https://www.trustpilot.com/review/www.priceline.com?languages=en&page=98
Crawling webpage: https://www.trustpilot.com/review/www.priceline.com?languages=en&page=99
Crawling webpage: https://www.trustpilot.com/review/www.priceline.com?languages=en&page=100
Crawling webpage: https://www.trustpilot.com/review/www.priceline.com?languages=en&page=101
Crawling webpage: https://www.trustpilot.com/review/www.priceline.com?languages=en&page=

In [6]:
# Check the number of data in each column to see if there is any data that does not match up
print(len(priceline_review_weekdays))
print(len(priceline_review_scores))
print(len(priceline_review_contents))
print(len(priceline_review_numbers))

2140
2140
2140
2140


In [7]:
# Assign SentimentIntensityAnalyzer() to variable analyzer
analyzer = SentimentIntensityAnalyzer()

# Create an empty list that is labeled priceline_compounds to collect valence values
priceline_compounds = []

# For loop to get the polarity scores of the review content and the valence
for priceline_review_content in priceline_review_contents:
    # Polarity score of the review content
    vs = analyzer.polarity_scores(priceline_review_content)
    # Determining valence of review content
    if vs['compound'] >= 0.5:
        priceline_compounds.append("positive")
    elif vs['compound'] <= -0.5:
        priceline_compounds.append("negative")
    else:
        priceline_compounds.append("neutral")

# Assign the valence values to the key sentiment_compound
priceline_reviews["sentiment_compound"] = priceline_compounds
# Check the number of data in priceline_compounds to check if it matches with review_contents
print(len(priceline_compounds))

2140


In [8]:
# Make a dataframe with the priceline_reviews dictionary
priceline_df = pd.DataFrame(priceline_reviews)
priceline_df

,review_contents,review_numbers,review_scores,review_weekdays,company_name,sentiment_compound
0,Priceline made everything so easy ...,1,5,Tuesday,priceline,positive
1,I’m a huge fan of priceline; when ...,1,5,Tuesday,priceline,neutral
2,The satisfaction is guaranteed for...,1,5,Tuesday,priceline,positive
3,Great services and prices,1,5,Tuesday,priceline,positive
4,"This hotel was very clean, spotles...",1,5,Tuesday,priceline,positive
5,Priceline was really user friendly...,3,5,Tuesday,priceline,positive
6,Love Priceline,1,5,Tuesday,priceline,positive
7,Good prices and easy to book rooms...,1,5,Tuesday,priceline,positive
8,Good. Nice. Pleasant. Met excpecta...,2,5,Monday,priceline,positive
9,Have used Priceline since its very...,3,5,Monday,priceline,neutral


In [9]:
# Save priceline_reviews dataframe to tsv file on laptop
priceline_df.to_csv("/Users/angieryu2202/Desktop/priceline_reviews_tp.tsv", sep="\t")

## 5. Hostelworld 

In [11]:
# Create an empty list to store hostelworld review urls
hostel_review_urls = []
# Create a url for each priceline review page (although there are actually 622 pages, downsampled to 107 to sort out the imbalance)
for x in range (0,107):
    url = "https://www.trustpilot.com/review/www.hostelworld.com?languages=en&page="+str(x+1)
    # Append the urls to the hostel_review_urls list
    hostel_review_urls.append(url)
    #print(url)
# Print the number of urls in the hostel_review_urls list
print(len(hostel_review_urls))

107


In [12]:
# Process indicator
process = "Crawling webpage: "

# Making empty dictionary for hostelworld reviews
hostel_reviews = {}

# Making empty lists for hostelworld review components
hostel_review_contents = []
hostel_review_scores = []
hostel_review_numbers = []
hostel_review_weekdays = []

for url in hostel_review_urls:
    
    html = urlopen(url)
    soup = BeautifulSoup(html,"html.parser")
    
    # Review content
    review_content_infos = soup.find_all("p",{"class":"review-info__body__text"})
    for review_content_info in review_content_infos:
        review_content_info_str = str(review_content_info)
        review_content = review_content_info_str.split('body__text">')[1].split('</p>')[0].replace("\n","").replace("<br/>","")
        hostel_review_contents.append(review_content)
    
    # Review score
    review_score_infos = soup.find_all("script",{"data-initial-state":"review-info"})
    for review_score_info in review_score_infos:
        review_score = review_score_info.text.split('stars":')[1].split("}")[0]
        hostel_review_scores.append(review_score)

    # Number of reviews the reviewer wrote
    review_number_infos = soup.find_all("span",{"class":"consumer-info__details__review-count"})
    for review_number_info in review_number_infos:
        review_number_info_str = str(review_number_info)
        review_number = review_number_info_str.split('review-count">')[1].split(' review')[0].replace("\n","")
        hostel_review_numbers.append(review_number)
    
    # Weekday of review
    review_date_infos = soup.find_all("div",{"class":"header__verified__date"})
    for review_date_info in review_date_infos:
        review_date_info_str = str(review_date_info)
        review_weekday = review_date_info_str.split('title="')[1].split(",")[0]
        hostel_review_weekdays.append(review_weekday)

    
    # Print process to know how your crawler is doing
    print(process + str(url))
    
# Add all the information collected in the four lists to the dictionary
hostel_reviews["review_contents"] = hostel_review_contents
hostel_reviews["review_numbers"] = hostel_review_numbers
hostel_reviews["review_scores"] = hostel_review_scores
hostel_reviews["review_weekdays"] = hostel_review_weekdays

# Add the company name "hostelworld" to company_name
hostel_reviews["company_name"] = "hostelworld"

Crawling webpage: https://www.trustpilot.com/review/www.hostelworld.com?languages=en&page=1
Crawling webpage: https://www.trustpilot.com/review/www.hostelworld.com?languages=en&page=2
Crawling webpage: https://www.trustpilot.com/review/www.hostelworld.com?languages=en&page=3
Crawling webpage: https://www.trustpilot.com/review/www.hostelworld.com?languages=en&page=4
Crawling webpage: https://www.trustpilot.com/review/www.hostelworld.com?languages=en&page=5
Crawling webpage: https://www.trustpilot.com/review/www.hostelworld.com?languages=en&page=6
Crawling webpage: https://www.trustpilot.com/review/www.hostelworld.com?languages=en&page=7
Crawling webpage: https://www.trustpilot.com/review/www.hostelworld.com?languages=en&page=8
Crawling webpage: https://www.trustpilot.com/review/www.hostelworld.com?languages=en&page=9
Crawling webpage: https://www.trustpilot.com/review/www.hostelworld.com?languages=en&page=10
Crawling webpage: https://www.trustpilot.com/review/www.hostelworld.com?languag

Crawling webpage: https://www.trustpilot.com/review/www.hostelworld.com?languages=en&page=90
Crawling webpage: https://www.trustpilot.com/review/www.hostelworld.com?languages=en&page=91
Crawling webpage: https://www.trustpilot.com/review/www.hostelworld.com?languages=en&page=92
Crawling webpage: https://www.trustpilot.com/review/www.hostelworld.com?languages=en&page=93
Crawling webpage: https://www.trustpilot.com/review/www.hostelworld.com?languages=en&page=94
Crawling webpage: https://www.trustpilot.com/review/www.hostelworld.com?languages=en&page=95
Crawling webpage: https://www.trustpilot.com/review/www.hostelworld.com?languages=en&page=96
Crawling webpage: https://www.trustpilot.com/review/www.hostelworld.com?languages=en&page=97
Crawling webpage: https://www.trustpilot.com/review/www.hostelworld.com?languages=en&page=98
Crawling webpage: https://www.trustpilot.com/review/www.hostelworld.com?languages=en&page=99
Crawling webpage: https://www.trustpilot.com/review/www.hostelworld.co

In [13]:
# Check the number of data in each column to see if there is any data that does not match up
print(len(hostel_review_weekdays))
print(len(hostel_review_scores))
print(len(hostel_review_contents))
print(len(hostel_review_numbers))

2140
2140
2140
2140


In [14]:
# Assign SentimentIntensityAnalyzer() to variable analyzer
analyzer = SentimentIntensityAnalyzer()

# Create an empty list that is labeled hostel_compounds to collect valence values
hostel_compounds = []

# For loop to get the polarity scores of the review content and the valence
for hostel_review_content in hostel_review_contents:
    # Polarity score of the review content
    vs = analyzer.polarity_scores(hostel_review_content)
    # Determining valence of review content
    if vs['compound'] >= 0.5:
        hostel_compounds.append("positive")
    elif vs['compound'] <= -0.5:
        hostel_compounds.append("negative")
    else:
        hostel_compounds.append("neutral")

# Assign the valence values to the key sentiment_compound
hostel_reviews["sentiment_compound"] = hostel_compounds
# Check the number of data in hostel_compounds to check if it matches with review_contents
print(len(hostel_compounds))

2140


In [16]:
# Make a dataframe with the hostel_reviews dictionary
hostel_df = pd.DataFrame(hostel_reviews)
hostel_df

,review_contents,review_numbers,review_scores,review_weekdays,company_name,sentiment_compound
0,I recently booked a hostel in Arge...,1,1,Sunday,hostelworld,negative
1,Zero or less would be the true rat...,1,1,Saturday,hostelworld,negative
2,"I posted here on September 13, 201...",2,1,Saturday,hostelworld,negative
3,"Update on Nov 28, 2018:HostelWorld...",6,1,Friday,hostelworld,negative
4,Very poor experience with this com...,1,1,Monday,hostelworld,neutral
5,HW used to be great back in the da...,6,3,Monday,hostelworld,positive
6,There is no customer service at al...,1,1,Thursday,hostelworld,negative
7,What can I say...the beds are grea...,1,5,Monday,hostelworld,positive
8,Absolute worst case scenario: I bo...,4,1,Monday,hostelworld,negative
9,This company allowed someone else ...,1,1,Wednesday,hostelworld,negative


In [17]:
# Save hostelworld_reviews dataframe to tsv file on laptop
hostel_df.to_csv("/Users/angieryu2202/Desktop/hostel_reviews_tp.tsv", sep="\t")